In [ ]:
from jupyterlab_h5web import H5Web

In [ ]:
import struct
a = struct.pack('I', 0x04034b50)
print(a)

In [ ]:
H5Web("jsmap.nxs")
H5Web("SuperScan (HADF).nxs")

In [ ]:
from IPython.display import JSON

In [ ]:
JSON("2022-02-18_Metadata_Kuehbach.nsproj")

In [ ]:
import yaml
import flatdict as fd
file_name = "2022-02-18_Metadata_Kuehbach.nsproj"
with open(file_name, 'r') as stream:
    yml = fd.FlatDict(yaml.safe_load(stream), delimiter='/')
    for serialized_path in yml.keys():
        print(f"{serialized_path}\t\t\t{yml[serialized_path]}")

1. Load nsproj walk display_items and compute data_alphabet_value check if such files exist
2. For each such file perform the mapping into an own NXevent_data_em instance
   For the H5 read attribute try to figure out what conceptually the item is

Reproducibly recover the uuid management choices from ns

In [ ]:
import uuid
def encode(uuid_: uuid.UUID, alphabet: str) -> str:
    result = str()
    uuid_int = uuid_.int
    while uuid_int:
        uuid_int, digit = divmod(uuid_int, len(alphabet))
        result += alphabet[digit]
    return result

In [ ]:
data_item_uuid_str = "e4900934-9332-4706-94cc-6fbe989647d9"
data_item_uuid_str = "72f1cda3-dea5-4a1b-bb85-ee34d5009860"
data_item_uuid_str = "8506c02c-8ba8-48ef-b22f-5e2878abe2d9"
data_item_uuid_str = "b753bea8-2526-4770-875d-52a262c3d9c4"
data_item_uuid_str = "7e19bb9e-78a5-4d1c-8bd3-b60e6cebae42"
# data_item_uuid_str = "9c48aa2a-c910-4200-a625-0f01a09a3578"
data_item_uuid_uuid = uuid.UUID(f'{data_item_uuid_str}')
print(f'data_{encode(data_item_uuid_uuid, "ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")}')  # 25 character results

In [ ]:
H5Web("data_7EPPSHNUFKH6F6A4JCR45J03G.h5")

In [ ]:
import h5py
fnm = "data_7EPPSHNUFKH6F6A4JCR45J03G.h5"
# fnm = "data_O4W29EVTX4DPTR33VRYHLI2QH.h5"
# fnm = "data_24ZGOFFSF7NVOP322TBCKF0YF.h5"
h5r = h5py.File(fnm, "r")
metadata = h5r["data"].attrs["properties"]
print(type(metadata))
h5r.close()
# with open("data_7EPPSHNUFKH6F6A4JCR45J03G.h5.metadata.json", "w") as f:
#     f.write(metadata)
import json
print(type(metadata))
json_object = json.loads(metadata)
print(type(json_object))
with open(f"{fnm}.metadata.json", "w") as f:
    json.dump(metadata, f, ensure_ascii=False)

In [ ]:
JSON(json_object)

In [ ]:
H5Web(fnm)

In [ ]:
for count, value in enumerate([0], start=1):
    print(count, value)

In [ ]:
new_shape = []
not new_shape

In [ ]:
import flatdict as fd
import numpy as np
import yaml
import re
import datetime
import pandas as pd
import os

# file://///wsl.localhost/Ubuntu/home/mkuehbach/SPRINT-8/nexus_code_camp_2022/parser-nexus/nexusparser/tools/yaml2nxdl/sprint9_update_nx_em/test_manual_building/nexus_definitions/build/manual/build/html/classes/contributed_definitions/NXem.html#nxem

# https://stackoverflow.com/questions/3663450/remove-substring-only-at-the-end-of-string
def rchop(s, suffix):
    if suffix and s.endswith(suffix):
        return s[:-len(suffix)]
    return s

# import metadata from nionswift json file
wkdir = os.getcwd()
file_names = [
    "MultiAcquire (Dectris ELA) #1.json",
    "MultiAcquire (HADF) #1.json",
    "Recording of SuperScan (BF).json",
    "Recording of SuperScan (HADF).json",
    "RonchiCam1.json",
    "RonchiCam2.json",
    "Spectrum Image EELS.json",
    "Spectrum Image HADF.json",
    "SuperScan (BF).json",
    "SuperScan (HADF).json",
    "SuperScan (MADF).json"
    ]
file_names = ["HAADF_01.json"]
# file_names = [f"{fnm}.metadata.json"]

# first of all create a global namespace of all keywords
nion_path_id = 1
all_uniq_path_to_id = {}
all_uniq_id_to_path = {}

for f in file_names:
    file_name = f"{wkdir}/{f}"
    print(file_name)

    with open(file_name, 'r') as stream:
        yml = fd.FlatDict(yaml.safe_load(stream), delimiter='/')

    for serialized_path in yml.keys():
        print(f"{serialized_path}\t\t\t{yml[serialized_path]}")

In [ ]:
    # nionswift metadata have the subtility that some paths have lists as values
    # which contain again dictionaries of the next deeper layer with metadata
    # these we also would like to flatten
    # nion_path_id = 1
    # ns_uniq_path_to_id = {}
    # ns_uniq_id_to_path = {}
    for path in yml.keys():
        prefix = path
        if isinstance(yml[prefix], list):
            only_dicts = True
            for obj in yml[prefix]:
                if type(obj) is not dict:
                    only_dicts = False
                # all obj have to be tested

            if only_dicts is True:
                for obj in yml[prefix]:
                    flat = fd.FlatDict(obj, delimiter='/')
                    for key, val in flat.items():
                        keyword = prefix + '/' + key
                        value = val
                        if keyword not in all_uniq_path_to_id.keys():
                            all_uniq_path_to_id[keyword] = (nion_path_id, file_name)
                            # ns_uniq_path_to_id[keyword] = (nion_path_id, value)
                            # ns_uniq_id_to_path[nion_path_id] = (keyword, value)
                            nion_path_id += 1
        # else:
        keyword = path
        value = yml[path]
        if keyword not in all_uniq_path_to_id.keys():
            all_uniq_path_to_id[keyword] = (nion_path_id, file_name)
            # ns_uniq_path_to_id[keyword] = (nion_path_id, value)
            # ns_uniq_id_to_path[nion_path_id] = (keyword, value)
            nion_path_id += 1

    # next file

# create a table for pandas to export to excel/odfpy
nion_path_id = []
nion_path_name = []
nion_path_file = []
for key, val in all_uniq_path_to_id.items():
    nion_path_id.append(val[0])
    nion_path_name.append(key)
    nion_path_file.append(val[1])

df_ns = pd.DataFrame(
    {'nion_path_id_nionswift': nion_path_id,
     'flattened_path_in_nionswift.json': nion_path_name})
df_ns = df_ns.sort_values(by=['flattened_path_in_nionswift.json'])
# 'file_name first found': c})

# remove temporary variables # all_uniq_id_to_path
variables = [
    file_name, file_names, flat, key, keyword, nion_path_file,\
    nion_path_id, nion_path_name, nx_em_id_to_path, nx_em_path_id, \
    nx_em_path_name, nx_em_path_to_id, obj, only_dicts, \
    path, prefix, serialized_path, stream, val, value]
for i in variables:
    del i